In [3]:
import sqlite3 as sql
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as ipyw
import pandas as pd
import country_converter as coco

In [5]:
def read_database(year, state="", age=999, female=999, urban=999, edu=999):
    # 999 is a placeholder value meaning "select all" for variables.
    # age = 999 means no filtering is applied, and data for all ages is selected.

    def execute_query(table_name, condition):
        # defines a SQL query to fetch data from the specified table.
        # the 'median' is normalized by dividing it by the average median value from the same table.
        query = f"SELECT iso3, median / (SELECT AVG(median) FROM {table_name}) AS median FROM {table_name} WHERE {condition}"

        # executes the SQL query and return the result as a pandas DataFrame
        return pd.read_sql_query(query, con)

    # opens the database connection using a context manager
    with sql.connect('cleaned_data.db') as con:
        # when no specific state is selected, define the condition to include all states
        if state == "":
            # crrates a condition string to filter the data by age, gender, urban/rural, education, and adjusted year
            condition = f"age = {age} AND female = {female} AND urban = {urban} AND edu = {edu} AND year = {year - year % 5}"
            # query incidence rates for the given year
            query_incidence = f"SELECT iso3, [{year}] AS incidence_rate FROM Incidence_rate"
        else:
            # when a specific state is selected, add the state filter to the condition
            condition = f"age = {age} AND female = {female} AND urban = {urban} AND edu = {edu} AND year = {year - year % 5} AND iso3='{state}'"
            # query incidence rates for the given year and state
            query_incidence = f"SELECT iso3, [{year}] AS incidence_rate FROM Incidence_rate WHERE iso3='{state}'"

        # fetchs normalized data for sugar, salt, and saturated fat using the execute_query function
        df_sugar = execute_query("Sugar", condition)  # fetch sugar consumption data
        df_salt = execute_query("Salt", condition)  # fetch salt consumption data
        df_saturated_fat = execute_query("Saturated_fat", condition)  # fetch saturated fat data
        df_incidence_rate = pd.read_sql_query(query_incidence, con)  # fetch incidence rate data

    # returns the resulting dataFrames for sugar, salt, saturated fat, and incidence rate
    return df_sugar, df_salt, df_saturated_fat, df_incidence_rate


In [7]:
def load_data(year, state=""):
    # call the read_database function to fetch data for sugar, salt, saturated fat, and incidence rate
    sugar, salt, fat, incidence_rate = read_database(year, state)

    # add a new column 'type' to indicate the type of data for each dataset
    sugar['type'] = 'sugar'  # marks the data as sugar-related
    salt['type'] = 'salt'  # marks the data as salt-related
    fat['type'] = 'saturated_fat'  # marks the data as saturated fat-related

    # merges each dataset with incidence_rate on the 'iso3' column (country code)
    sugar = pd.merge(sugar, incidence_rate, on='iso3')  # combines sugar data with incidence rates
    salt = pd.merge(salt, incidence_rate, on='iso3')  # combines salt data with incidence rates
    fat = pd.merge(fat, incidence_rate, on='iso3')  # combines saturated fat data with incidence rates

    # concatenate all the datasets (sugar, salt, and fat) into one DataFrame
    data = pd.concat([sugar, salt, fat])  # combines all datasets vertically

        

    # returns the final concatenated DataFrame
    return data


In [11]:
state = load_data(2004)
mask = state['type'] == 'sugar'
state = pd.DataFrame(state[mask])

#f= pd.Series(state['iso3'],name = 'country')
#cc = coco.CountryConverter()
#country_names = cc.pandas_convert(series=f, to='name_short')  
#country_names = pd.DataFrame(country_names)

country_names = state['iso3']


country =  ['']
for i in range(len(country_names)):
    country = country +[(state['iso3'])[i]]


In [13]:
from ipywidgets import interact, FloatSlider ,Dropdown
from IPython.display import display, clear_output

sns.set_theme(style="white")

def plot_interactive(year, state=country):

  clear_output(wait=True)

  #plt.figure(figsize=(8, 5))
  if state == "":

    data = load_data(year, state ="" )
    fig = sns.relplot(x="median", y="incidence_rate", hue="type",
              sizes=(40, 400), alpha=.5, palette="muted",
              height=6, data=data)
    plt.xlabel("Median of Nutrition Consumption (divided by the global average)")
    plt.ylabel("Incidence Rate of Cardiovascular diseases")
    #plt.show()
    
  else:

    all_data = pd.DataFrame()

    for i in range(1990, 2015):
        state =  coco.convert(names=state, to='iso3')

        data = load_data(i,state)
        data['year'] = i
        all_data = pd.concat([all_data, data])

    fig = sns.lineplot(x="year", y="median",
                hue="type",
                data=all_data)
    

    plt.xlabel("years")
    plt.ylabel("Median of Nutrition Consumption")
    #plt.show()
interact(plot_interactive, year=(1990, 2015, 1))
interact.toolbar_visible = False

interactive(children=(IntSlider(value=2002, description='year', max=2015, min=1990), Dropdown(description='sta…